In [1]:
pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 43.9 MB/s eta 0:00:00


In [2]:
import random

In [3]:
pip install gurobipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 28.5 MB/s eta 0:00:00


In [5]:
from gurobipy import *

# Conjuntos
S = range(1, 20)  # Set of possible facility locations
D = range(1, 11)  # Set of demand points
T = [1, 2, 3]  # Set of facility sizes

# Parámetros
C = {(s, t): 1000 + 500 * t for s in S for t in T}  # Cost of building a facility of size t at location s
Dj = {j: 100 for j in D}  # Demand of point j
B = 100000  # Maximum budget
d = {(s, s2): 10 for s in S for s2 in S}  # Distance between facilities s and s'
Dm = 20  # Maximum distance between facilities
Q = {t: 500 * t for t in T}  # Capacity of a facility of size t
r = {(s, j): 1 for s in S for j in D}  # 1 if facility s is within a fixed minimum distance from demand point j, 0 otherwise

# Crear modelo
m = Model('Facility Location')

In [6]:
# Crear variables de decisión
x = m.addVars(S, T, vtype=GRB.BINARY, name="x")  # 1 if facility of size t is built at location s
y = m.addVars(S, D, vtype=GRB.BINARY, name="y")  # 1 if facility s is assigned to serve demand point j

# Función objetivo: maximizar la demanda total servida por las instalaciones
m.setObjective(quicksum(Dj[j] * quicksum(y[s, j] for s in S) for j in D), GRB.MAXIMIZE)

# Restricción del presupuesto
m.addConstr(quicksum(C[s, t] * x[s, t] for s in S for t in T) <= B, name="Budget")

# Restricción de asignación
m.addConstrs(quicksum(y[s, j] for s in S) <= 1 for j in D)

# Restricción de distancia mínima
m.addConstrs(y[s, j] <= r[s, j] for s in S for j in D)

# Restricción de capacidad
m.addConstrs(quicksum(Dj[j] * y[s, j] for j in D) <= Q[t] * x[s, t] for s in S for t in T)

# Restricción de distancia máxima entre instalaciones
#m.addConstrs(quicksum((d[s, s2] * x[s, t] * x[s2, t2]) for s in S for s2 in S for t in T for t2 in T) <= Dm ** 2)

# Vinculación entre las variables de asignación y construcción de instalaciones
m.addConstrs(y[s, j] <= x[s, t] for s in S for j in D for t in T)


{(1, 1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7, 1): <gurobi.Constr *Awaiting Model Upd

In [7]:
# Optimizar modelo
m.optimize()


Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 828 rows, 247 columns and 2204 nonzeros
Model fingerprint: 0x6857fef0
Variable types: 0 continuous, 247 integer (247 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+02, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+05]
Found heuristic solution: objective -0.0000000
Presolve removed 228 rows and 0 columns
Presolve time: 0.01s
Presolved: 600 rows, 247 columns, 1596 nonzeros
Variable types: 0 continuous, 247 integer (247 binary)
Found heuristic solution: objective 1000.0000000

Root relaxation: cutoff, 0 iterations, 0.00 seconds (0.00 work units)

Explored 1 nodes (0 simplex iterations) in 0.04 seconds (0.01 work units)
Thread count was 2 (of 2 available processors)

Solution count 2: 1000 -0 


In [11]:
# Imprimir resultado
if m.status == GRB.OPTIMAL:
    print(f"Objetivo: {m.objVal:.2f}")
    print("Instalaciones construidas:")
    for s in S:
        for t in T:
            if x[s, t].x > 0.5:
                print(f"Facility {s} of size {t} is built.")
                print("Demandas atendidas:")
                for j in D:
                  for s in S:
                    if y[s, j].x > 0.5:
                      print(f"Demand point {j} is served by facility {s}.")
                    #else:
                      #print("El modelo no pudo ser resuelto.")



Objetivo: 1000.00
Instalaciones construidas:
Facility 1 of size 1 is built.
Demandas atendidas:
Demand point 1 is served by facility 1.
Demand point 2 is served by facility 1.
Demand point 3 is served by facility 1.
Demand point 4 is served by facility 1.
Demand point 5 is served by facility 1.
Demand point 6 is served by facility 2.
Demand point 7 is served by facility 2.
Demand point 8 is served by facility 2.
Demand point 9 is served by facility 2.
Demand point 10 is served by facility 2.
Facility 2 of size 1 is built.
Demandas atendidas:
Demand point 1 is served by facility 1.
Demand point 2 is served by facility 1.
Demand point 3 is served by facility 1.
Demand point 4 is served by facility 1.
Demand point 5 is served by facility 1.
Demand point 6 is served by facility 2.
Demand point 7 is served by facility 2.
Demand point 8 is served by facility 2.
Demand point 9 is served by facility 2.
Demand point 10 is served by facility 2.
